# ENGINEERING CALCULATION SHEET

**Project:** {{project_name}}  
**Project No:** {{project_number}}  
**Subject:** Lime Softening Design Calculation  
**Calc ID:** {{calc_id}}  
**Revision:** {{revision}}  
**Date:** {{date}}  

| Prepared By | Checked By | Approved By |
|------------|------------|-------------|
| {{preparer}} | {{checker}} | {{approver}} |
| {{prep_date}} | {{check_date}} | {{approve_date}} |

In [ ]:
# Papermill parameters
project_name = "Example Water Treatment Plant"
project_number = "2024-001"
calc_id = "CALC-LS-001"
revision = "0"
date = "2024-01-15"
preparer = "AI Assistant"
checker = "Senior Engineer"
approver = "Project Manager"
prep_date = "2024-01-15"
check_date = "-"
approve_date = "-"

# Calculation data
calculation_data = {}

## 1. OBJECTIVE

To determine the optimal lime (Ca(OH)₂) dosage for softening the raw water to achieve:
- Total hardness < 50 mg/L as CaCO₃
- Compliance with drinking water standards
- Minimal chemical usage and sludge production
- Practical design for full-scale implementation

## 2. DESIGN CRITERIA

- **Design Flow Rate:** 100 m³/h (2,400 m³/d)
- **Target Hardness:** < 50 mg/L as CaCO₃
- **Operating pH:** 10.0 - 10.5
- **Contact Time:** 30-60 minutes
- **Temperature:** 20°C
- **Settling Velocity:** 1.5 m/h (design basis)

## 3. REFERENCES

1. AWWA/ASCE Water Treatment Plant Design, 5th Edition
2. MWH Water Treatment: Principles and Design, 3rd Edition  
3. PHREEQC Version 3.8.6 - USGS
4. Standard Methods for Examination of Water and Wastewater, 23rd Edition
5. EPA Guidance Manual - Lime Softening (EPA 625/6-91/032)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import display, HTML, Markdown
from datetime import datetime

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Helper functions for safe data extraction
def safe_get(data, path, default=0):
    """Safely extract nested dictionary values"""
    keys = path.split('.')
    value = data
    for key in keys:
        if isinstance(value, dict):
            value = value.get(key, default)
        else:
            return default
    return value if value is not None else default

def format_number(value, decimals=2, default='N/A'):
    """Format number with fallback for None/NaN values"""
    try:
        if value is None or (isinstance(value, float) and np.isnan(value)):
            return default
        return f"{float(value):.{decimals}f}"
    except:
        return default

## 4. DATA STRUCTURE INSPECTION

### Understanding the calculation_data structure for proper extraction

In [ ]:
# Debug: Inspect data structure
print("=== CALCULATION DATA STRUCTURE ===")
print(f"Top-level keys: {list(calculation_data.keys())}")

# Inspect speciation results
if 'speciation_results' in calculation_data:
    spec_results = calculation_data['speciation_results']
    print(f"\nSpeciation results keys: {list(spec_results.keys())}")
    if 'solution_summary' in spec_results:
        print(f"  Solution summary keys: {list(spec_results['solution_summary'].keys())}")

# Inspect treatment results
if 'treatment_results' in calculation_data:
    treat_results = calculation_data['treatment_results']
    print(f"\nTreatment results keys: {list(treat_results.keys())}")
    if 'solution_summary' in treat_results:
        print(f"  Solution summary keys: {list(treat_results['solution_summary'].keys())}")
    if 'phases' in treat_results:
        print(f"  Phases keys: {list(treat_results.get('phases', {}).keys())}")

# Extract actual dosage information
if 'inputs' in calculation_data:
    print(f"\nInput data keys: {list(calculation_data['inputs'].keys())}")
    if 'reactants' in calculation_data['inputs']:
        print(f"  Reactants: {calculation_data['inputs']['reactants']}")

## 5. INPUT WATER QUALITY

In [ ]:
# Extract input water quality with robust error handling
spec_results = calculation_data.get('speciation_results', {})
initial_solution = spec_results.get('solution_summary', {})
initial_composition = initial_solution.get('composition', {})

# Also check inputs for initial analysis
input_analysis = calculation_data.get('inputs', {}).get('analysis', {})

# Merge data sources, preferring speciation results
ca_mmol = safe_get(initial_composition, 'Ca', safe_get(input_analysis, 'Ca', 0))
mg_mmol = safe_get(initial_composition, 'Mg', safe_get(input_analysis, 'Mg', 0))
alk_mmol = safe_get(initial_composition, 'C(4)', safe_get(input_analysis, 'Alkalinity', 0))
ph_value = safe_get(initial_solution, 'pH', safe_get(input_analysis, 'pH', 7.0))
temp_c = safe_get(initial_solution, 'temp', 20)

# Additional parameters
na_mmol = safe_get(initial_composition, 'Na', 0)
cl_mmol = safe_get(initial_composition, 'Cl', 0)
so4_mmol = safe_get(initial_composition, 'S(6)', 0)

# Create formatted table with proper units
parameters = [
    {
        'Parameter': 'Calcium (Ca²⁺)',
        'Concentration': format_number(ca_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': format_number(ca_mmol * 40.08, 1),
        'As mg/L CaCO₃': format_number(ca_mmol * 50.04, 1)
    },
    {
        'Parameter': 'Magnesium (Mg²⁺)',
        'Concentration': format_number(mg_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': format_number(mg_mmol * 24.31, 1),
        'As mg/L CaCO₃': format_number(mg_mmol * 50.04 * (40.08/24.31), 1)
    },
    {
        'Parameter': 'Alkalinity',
        'Concentration': format_number(alk_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': '-',
        'As mg/L CaCO₃': format_number(alk_mmol * 50.04, 1)
    },
    {
        'Parameter': 'pH',
        'Concentration': format_number(ph_value, 2),
        'Unit': '-',
        'As mg/L': '-',
        'As mg/L CaCO₃': '-'
    },
    {
        'Parameter': 'Temperature',
        'Concentration': format_number(temp_c, 1),
        'Unit': '°C',
        'As mg/L': '-',
        'As mg/L CaCO₃': '-'
    }
]

# Add additional parameters if present
if na_mmol > 0:
    parameters.append({
        'Parameter': 'Sodium (Na⁺)',
        'Concentration': format_number(na_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': format_number(na_mmol * 22.99, 1),
        'As mg/L CaCO₃': '-'
    })

if cl_mmol > 0:
    parameters.append({
        'Parameter': 'Chloride (Cl⁻)',
        'Concentration': format_number(cl_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': format_number(cl_mmol * 35.45, 1),
        'As mg/L CaCO₃': '-'
    })

if so4_mmol > 0:
    parameters.append({
        'Parameter': 'Sulfate (SO₄²⁻)',
        'Concentration': format_number(so4_mmol, 2),
        'Unit': 'mmol/L',
        'As mg/L': format_number(so4_mmol * 96.06, 1),
        'As mg/L CaCO₃': '-'
    })

input_df = pd.DataFrame(parameters)
display(HTML("<b>Table 1: Raw Water Quality Analysis</b>"))
display(input_df.style.hide(axis='index').set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'left'), ('padding', '8px')]}
]))

## 6. METHODOLOGY

### 6.1 Lime Softening Chemistry

The fundamental reactions for lime softening are:

**Carbonate Hardness Removal:**
$$\text{Ca}^{2+} + 2\text{HCO}_3^- + \text{Ca(OH)}_2 \rightarrow 2\text{CaCO}_3 \downarrow + 2\text{H}_2\text{O}$$

**Magnesium Removal:**
$$\text{Mg}^{2+} + \text{Ca(OH)}_2 \rightarrow \text{Mg(OH)}_2 \downarrow + \text{Ca}^{2+}$$

**Non-Carbonate Hardness (if soda ash added):**
$$\text{Ca}^{2+} + \text{Na}_2\text{CO}_3 \rightarrow \text{CaCO}_3 \downarrow + 2\text{Na}^+$$

### 6.2 Design Approach

1. Calculate carbonate and non-carbonate hardness
2. Determine theoretical lime requirement
3. Apply practical excess factor (10-20%)
4. Model using PHREEQC for equilibrium
5. Evaluate precipitation kinetics
6. Calculate sludge production and handling requirements
7. Determine chemical costs and consumption rates

## 7. CALCULATIONS

### 7.1 Hardness Classification

In [ ]:
# Calculate hardness in mg/L as CaCO3
ca_hardness = ca_mmol * 50.04  # Ca as CaCO3
mg_hardness = mg_mmol * 50.04 * (40.08/24.31)  # Mg as CaCO3 (correction factor)
total_hardness = ca_hardness + mg_hardness
alkalinity_caco3 = alk_mmol * 50.04

# Determine carbonate and non-carbonate hardness
if total_hardness <= alkalinity_caco3:
    carbonate_hardness = total_hardness
    non_carbonate_hardness = 0
else:
    carbonate_hardness = alkalinity_caco3
    non_carbonate_hardness = total_hardness - alkalinity_caco3

print("=== HARDNESS ANALYSIS ===")
print(f"Calcium Hardness:        {ca_hardness:.1f} mg/L as CaCO₃")
print(f"Magnesium Hardness:      {mg_hardness:.1f} mg/L as CaCO₃")
print(f"Total Hardness:          {total_hardness:.1f} mg/L as CaCO₃")
print(f"Alkalinity:              {alkalinity_caco3:.1f} mg/L as CaCO₃")
print(f"\nCarbonate Hardness:      {carbonate_hardness:.1f} mg/L as CaCO₃")
print(f"Non-Carbonate Hardness:  {non_carbonate_hardness:.1f} mg/L as CaCO₃")

# Water classification
if total_hardness < 60:
    hardness_class = "Soft"
elif total_hardness < 120:
    hardness_class = "Moderately Hard"
elif total_hardness < 180:
    hardness_class = "Hard"
else:
    hardness_class = "Very Hard"
    
print(f"\nWater Classification:    {hardness_class}")

### 7.2 Theoretical Lime Requirement

In [ ]:
# Theoretical lime calculations based on stoichiometry
print("=== THEORETICAL LIME REQUIREMENT ===")

# 1. Lime for carbonate hardness (CO2 + bicarbonate alkalinity)
# CO2 + Ca(OH)2 → CaCO3 + H2O
# HCO3- + Ca(OH)2 → CaCO3 + H2O + OH-
co2_mmol = 0.5  # Assumed free CO2 (typical for groundwater)
lime_co2 = co2_mmol
lime_carb = alk_mmol  # 1:1 with alkalinity
print(f"For CO2 neutralization:   {lime_co2:.2f} mmol/L")
print(f"For carbonate hardness:   {lime_carb:.2f} mmol/L")

# 2. Lime for magnesium precipitation
# Mg2+ + Ca(OH)2 → Mg(OH)2 + Ca2+
lime_mg = mg_mmol  # 1:1 with Mg
print(f"For magnesium removal:    {lime_mg:.2f} mmol/L")

# 3. Excess lime for pH adjustment to 10.3-10.5
# Empirical: 0.3-0.5 mmol/L for pH adjustment
lime_pH = 0.5
print(f"For pH adjustment:        {lime_pH:.2f} mmol/L")

# Total theoretical requirement
lime_theoretical = lime_co2 + lime_carb + lime_mg + lime_pH
print(f"\nTotal theoretical:        {lime_theoretical:.2f} mmol/L")
print(f"                          {lime_theoretical * 74.09:.1f} mg/L Ca(OH)₂")

# Apply practical excess factor
excess_factor = 1.15  # 15% excess for operational flexibility
lime_practical = lime_theoretical * excess_factor
print(f"\nWith {(excess_factor-1)*100:.0f}% excess:           {lime_practical:.2f} mmol/L")
print(f"                          {lime_practical * 74.09:.1f} mg/L Ca(OH)₂")

# Store the designed lime dose
designed_lime_dose_mmol = lime_practical

### 7.3 PHREEQC Equilibrium Modeling Results

In [ ]:
# Extract treatment results with proper error handling
treatment_results = calculation_data.get('treatment_results', {})
solution_summary = treatment_results.get('solution_summary', {})
final_composition = solution_summary.get('composition', {})

# Extract the actual lime dose used from inputs
actual_lime_dose = 0
if 'inputs' in calculation_data:
    reactants = calculation_data['inputs'].get('reactants', [])
    for reactant in reactants:
        if reactant.get('formula') == 'Ca(OH)2':
            actual_lime_dose = reactant.get('amount', 0)
            break

print("=== PHREEQC SIMULATION RESULTS ===")
print(f"Lime dose applied:        {actual_lime_dose:.2f} mmol/L")
print(f"                          {actual_lime_dose * 74.09:.1f} mg/L Ca(OH)₂")
print(f"\nFinal Solution Properties:")
print(f"  pH:                     {safe_get(solution_summary, 'pH', 'N/A')}")
print(f"  Temperature:            {safe_get(solution_summary, 'temp', 20)}°C")
print(f"  Ionic strength:         {safe_get(solution_summary, 'ionic_strength', 0):.4f} mol/L")
print(f"  Electrical balance:     {safe_get(solution_summary, 'percent_error', 0):.2f}%")

# Extract precipitated phases correctly
phases = treatment_results.get('phases', {})
precipitated_minerals = []
total_precipitate_g_L = 0

print("\nPrecipitated Minerals:")
print(f"{'Mineral':<20} {'SI':>8} {'Amount (mol/L)':>15} {'Mass (g/L)':>12}")
print("-" * 60)

# Mineral molecular weights
mineral_mw = {
    'Calcite': 100.09,
    'Aragonite': 100.09,
    'Brucite': 58.32,
    'Mg(OH)2': 58.32,
    'Dolomite': 184.40,
    'Portlandite': 74.09
}

for mineral, data in phases.items():
    if isinstance(data, dict):
        si = data.get('si', -999)
        amount = data.get('amount', 0)
        if si > 0 and amount > 0:  # Supersaturated and precipitated
            mw = mineral_mw.get(mineral, 100)
            mass_g_L = amount * mw
            print(f"{mineral:<20} {si:>8.2f} {amount:>15.4f} {mass_g_L:>12.2f}")
            precipitated_minerals.append({
                'mineral': mineral,
                'amount_mol': amount,
                'mass_g': mass_g_L
            })
            total_precipitate_g_L += mass_g_L

# If no precipitates found in phases, estimate from mass balance
if not precipitated_minerals:
    print("\nNote: Precipitate data not found in phases. Estimating from mass balance...")
    
    # Calculate removal
    ca_removed = ca_mmol - safe_get(final_composition, 'Ca', 0)
    mg_removed = mg_mmol - safe_get(final_composition, 'Mg', 0)
    
    if ca_removed > 0:
        calcite_mol = ca_removed
        calcite_g = calcite_mol * 100.09 / 1000  # Convert to g/L
        print(f"{'Calcite (estimated)':<20} {'N/A':>8} {calcite_mol/1000:>15.4f} {calcite_g:>12.2f}")
        total_precipitate_g_L += calcite_g
        
    if mg_removed > 0:
        brucite_mol = mg_removed
        brucite_g = brucite_mol * 58.32 / 1000  # Convert to g/L
        print(f"{'Brucite (estimated)':<20} {'N/A':>8} {brucite_mol/1000:>15.4f} {brucite_g:>12.2f}")
        total_precipitate_g_L += brucite_g

print(f"\nTotal precipitate:        {total_precipitate_g_L:.2f} g/L")
print(f"Sludge volume (3% solids): {total_precipitate_g_L/30:.1f} L/m³ water")

### 7.4 Final Water Quality

In [ ]:
# Extract final water quality
final_ca = safe_get(final_composition, 'Ca', 0)
final_mg = safe_get(final_composition, 'Mg', 0)
final_alk = safe_get(final_composition, 'C(4)', 0)
final_ph = safe_get(solution_summary, 'pH', 0)

# Calculate final hardness
final_ca_hardness = final_ca * 50.04
final_mg_hardness = final_mg * 50.04 * (40.08/24.31)
final_hardness = final_ca_hardness + final_mg_hardness

# Create comparison table
comparison_data = [
    {
        'Parameter': 'pH',
        'Initial': format_number(ph_value, 2),
        'Final': format_number(final_ph, 2),
        'Target': '10.0-10.5',
        'Unit': '-',
        'Status': '✓' if 10.0 <= final_ph <= 10.5 else '✗'
    },
    {
        'Parameter': 'Total Hardness',
        'Initial': format_number(total_hardness, 0),
        'Final': format_number(final_hardness, 0),
        'Target': '<50',
        'Unit': 'mg/L CaCO₃',
        'Status': '✓' if final_hardness < 50 else '✗'
    },
    {
        'Parameter': 'Calcium',
        'Initial': format_number(ca_mmol * 40.08, 0),
        'Final': format_number(final_ca * 40.08, 0),
        'Target': '-',
        'Unit': 'mg/L',
        'Status': '-'
    },
    {
        'Parameter': 'Magnesium',
        'Initial': format_number(mg_mmol * 24.31, 0),
        'Final': format_number(final_mg * 24.31, 0),
        'Target': '-',
        'Unit': 'mg/L',
        'Status': '-'
    },
    {
        'Parameter': 'Alkalinity',
        'Initial': format_number(alkalinity_caco3, 0),
        'Final': format_number(final_alk * 50.04, 0),
        'Target': '-',
        'Unit': 'mg/L CaCO₃',
        'Status': '-'
    }
]

comp_df = pd.DataFrame(comparison_data)
display(HTML("<b>Table 2: Water Quality Comparison</b>"))
display(comp_df.style.hide(axis='index').set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '8px')]}
]))

# Calculate removal efficiencies
if total_hardness > 0:
    hardness_removal = (total_hardness - final_hardness) / total_hardness * 100
    ca_removal = (ca_mmol - final_ca) / ca_mmol * 100 if ca_mmol > 0 else 0
    mg_removal = (mg_mmol - final_mg) / mg_mmol * 100 if mg_mmol > 0 else 0
    
    print(f"\n=== REMOVAL EFFICIENCIES ===")
    print(f"Total Hardness Removal:   {hardness_removal:.1f}%")
    print(f"Calcium Removal:          {ca_removal:.1f}%")
    print(f"Magnesium Removal:        {mg_removal:.1f}%")

### 7.5 Precipitation Kinetics Analysis

#### Time-dependent precipitation modeling for reactor design

In [ ]:
# Kinetic modeling for CaCO3 and Mg(OH)2 precipitation
print("=== PRECIPITATION KINETICS ===")

# Kinetic parameters (from literature)
# CaCO3: pseudo-first order, k = 0.1-0.5 min⁻¹ at pH 10-11
# Mg(OH)2: slower, k = 0.05-0.2 min⁻¹ at pH 10.5

k_caco3 = 0.3  # min⁻¹ (conservative design value)
k_mgoh2 = 0.1  # min⁻¹ (conservative design value)

# Time array (0 to 120 minutes)
time = np.linspace(0, 120, 121)

# Calculate concentrations vs time (first-order kinetics)
# C(t) = C0 * exp(-k*t)
ca_initial = ca_mmol
mg_initial = mg_mmol
ca_equilibrium = final_ca
mg_equilibrium = final_mg

# Precipitable amounts
ca_precipitable = ca_initial - ca_equilibrium
mg_precipitable = mg_initial - mg_equilibrium

# Time-dependent concentrations
ca_t = ca_equilibrium + ca_precipitable * np.exp(-k_caco3 * time)
mg_t = mg_equilibrium + mg_precipitable * np.exp(-k_mgoh2 * time)

# Precipitation completion
ca_precip_pct = (1 - np.exp(-k_caco3 * time)) * 100
mg_precip_pct = (1 - np.exp(-k_mgoh2 * time)) * 100

# Plot kinetics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Concentration vs time
ax1.plot(time, ca_t * 40.08, 'b-', linewidth=2, label='Ca²⁺')
ax1.plot(time, mg_t * 24.31, 'r-', linewidth=2, label='Mg²⁺')
ax1.axhline(y=ca_equilibrium * 40.08, color='b', linestyle='--', alpha=0.5, label='Ca²⁺ equilibrium')
ax1.axhline(y=mg_equilibrium * 24.31, color='r', linestyle='--', alpha=0.5, label='Mg²⁺ equilibrium')
ax1.set_xlabel('Time (minutes)', fontsize=12)
ax1.set_ylabel('Concentration (mg/L)', fontsize=12)
ax1.set_title('Precipitation Kinetics - Ion Concentrations', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Precipitation completion vs time
ax2.plot(time, ca_precip_pct, 'b-', linewidth=2, label='CaCO₃ precipitation')
ax2.plot(time, mg_precip_pct, 'r-', linewidth=2, label='Mg(OH)₂ precipitation')
ax2.axhline(y=90, color='g', linestyle='--', alpha=0.5, label='90% completion')
ax2.axvline(x=30, color='orange', linestyle='--', alpha=0.5, label='30 min detention')
ax2.axvline(x=60, color='purple', linestyle='--', alpha=0.5, label='60 min detention')
ax2.set_xlabel('Time (minutes)', fontsize=12)
ax2.set_ylabel('Precipitation Completion (%)', fontsize=12)
ax2.set_title('Precipitation Kinetics - Completion', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(0, 105)

plt.tight_layout()
plt.show()

# Calculate detention times for different completion levels
t_90_ca = -np.log(0.1) / k_caco3
t_90_mg = -np.log(0.1) / k_mgoh2
t_95_ca = -np.log(0.05) / k_caco3
t_95_mg = -np.log(0.05) / k_mgoh2

print(f"\nDETENTION TIME REQUIREMENTS:")
print(f"For 90% CaCO₃ precipitation:    {t_90_ca:.0f} minutes")
print(f"For 90% Mg(OH)₂ precipitation:  {t_90_mg:.0f} minutes")
print(f"For 95% CaCO₃ precipitation:    {t_95_ca:.0f} minutes")
print(f"For 95% Mg(OH)₂ precipitation:  {t_95_mg:.0f} minutes")
print(f"\nRecommended detention time:     {max(t_90_mg, 30):.0f} minutes (controlling: Mg(OH)₂)")

# Completion at standard detention times
print(f"\nCOMPLETION AT STANDARD DETENTION TIMES:")
for t in [15, 30, 45, 60]:
    ca_comp = (1 - np.exp(-k_caco3 * t)) * 100
    mg_comp = (1 - np.exp(-k_mgoh2 * t)) * 100
    print(f"At {t:2d} minutes: CaCO₃ = {ca_comp:5.1f}%, Mg(OH)₂ = {mg_comp:5.1f}%")

## 8. DESIGN SUMMARY

In [ ]:
# Design calculations
flow_rate = 100  # m³/h (from design criteria)
detention_time = max(t_90_mg, 30)  # minutes
reactor_volume = flow_rate * detention_time / 60  # m³

# Chemical consumption
lime_dose_kg_h = actual_lime_dose * 74.09 * flow_rate / 1000
lime_dose_kg_d = lime_dose_kg_h * 24
lime_dose_ton_y = lime_dose_kg_d * 365 / 1000

# Sludge production
sludge_dry_kg_h = total_precipitate_g_L * flow_rate
sludge_dry_kg_d = sludge_dry_kg_h * 24
sludge_volume_m3_d = sludge_dry_kg_d / 30  # at 3% solids

# Chemical costs (typical values)
lime_cost_usd_ton = 150  # $/ton
daily_chem_cost = lime_dose_kg_d * lime_cost_usd_ton / 1000
annual_chem_cost = daily_chem_cost * 365

design_box = f"""
<div style="border: 2px solid black; padding: 15px; margin: 20px 0; background-color: #f5f5f5;">
<h3 style="margin-top: 0;">DESIGN PARAMETERS</h3>
<table style="width: 100%; border-collapse: collapse;">
<tr><td colspan="2" style="padding: 5px; background-color: #e0e0e0;"><b>HYDRAULIC DESIGN</b></td></tr>
<tr><td style="padding: 5px;">Design Flow Rate:</td><td style="text-align: right;">{flow_rate} m³/h ({flow_rate*24:,} m³/d)</td></tr>
<tr><td style="padding: 5px;">Detention Time Required:</td><td style="text-align: right;">{detention_time:.0f} minutes</td></tr>
<tr><td style="padding: 5px;">Reactor Volume:</td><td style="text-align: right;">{reactor_volume:.0f} m³</td></tr>
<tr><td style="padding: 5px;">Number of Trains:</td><td style="text-align: right;">2 (duty + standby)</td></tr>
<tr><td colspan="2" style="padding: 5px; background-color: #e0e0e0;"><b>CHEMICAL DOSING</b></td></tr>
<tr><td style="padding: 5px;">Lime Dose:</td><td style="text-align: right;">{actual_lime_dose * 74.09:.0f} mg/L Ca(OH)₂</td></tr>
<tr><td style="padding: 5px;">Lime Consumption:</td><td style="text-align: right;">{lime_dose_kg_h:.1f} kg/h</td></tr>
<tr><td style="padding: 5px;">Daily Lime Usage:</td><td style="text-align: right;">{lime_dose_kg_d:,.0f} kg/day</td></tr>
<tr><td style="padding: 5px;">Annual Lime Usage:</td><td style="text-align: right;">{lime_dose_ton_y:,.0f} tons/year</td></tr>
<tr><td colspan="2" style="padding: 5px; background-color: #e0e0e0;"><b>SLUDGE PRODUCTION</b></td></tr>
<tr><td style="padding: 5px;">Dry Sludge Production:</td><td style="text-align: right;">{sludge_dry_kg_h:.1f} kg/h</td></tr>
<tr><td style="padding: 5px;">Daily Dry Sludge:</td><td style="text-align: right;">{sludge_dry_kg_d:,.0f} kg/day</td></tr>
<tr><td style="padding: 5px;">Wet Sludge Volume (3%):</td><td style="text-align: right;">{sludge_volume_m3_d:.1f} m³/day</td></tr>
<tr><td colspan="2" style="padding: 5px; background-color: #e0e0e0;"><b>OPERATIONAL COSTS</b></td></tr>
<tr><td style="padding: 5px;">Lime Cost (${lime_cost_usd_ton}/ton):</td><td style="text-align: right;">${daily_chem_cost:,.0f}/day</td></tr>
<tr><td style="padding: 5px;">Annual Chemical Cost:</td><td style="text-align: right;">${annual_chem_cost:,.0f}/year</td></tr>
<tr><td style="padding: 5px;">Unit Chemical Cost:</td><td style="text-align: right;">${daily_chem_cost/(flow_rate*24):.3f}/m³</td></tr>
</table>
</div>
"""
display(HTML(design_box))

## 9. EQUIPMENT SPECIFICATIONS

In [ ]:
# Equipment sizing calculations
equipment_specs = f"""
<div style="border: 1px solid #ccc; padding: 15px; margin: 20px 0;">
<h3>MAJOR EQUIPMENT LIST</h3>

<b>1. RAPID MIX TANKS (2 units)</b>
   - Volume: {reactor_volume * 0.1:.0f} m³ each
   - Detention time: 3 minutes
   - Mixing energy: 500-750 W/m³
   - Material: Concrete with epoxy lining

<b>2. FLOCCULATION BASINS (2 trains)</b>
   - Volume: {reactor_volume * 0.5:.0f} m³ per train
   - Detention time: {detention_time:.0f} minutes
   - Mixing energy: 50-75 W/m³ (tapered)
   - Number of stages: 3

<b>3. CLARIFIERS (2 units)</b>
   - Type: Solids contact clarifier
   - Surface area: {flow_rate/1.5:.0f} m² each
   - Surface loading: 1.5 m/h
   - Sludge recirculation: 2-5% of flow

<b>4. LIME FEED SYSTEM</b>
   - Type: Dry feed with slaking
   - Capacity: {lime_dose_kg_h * 1.2:.0f} kg/h (20% excess)
   - Number of feeders: 2 (duty + standby)
   - Slaker capacity: {lime_dose_kg_h * 1.5:.0f} kg/h
   - Day tank: {lime_dose_kg_d * 1.5:.0f} L (12-hour supply)

<b>5. SLUDGE HANDLING</b>
   - Thickener: Gravity type, {sludge_volume_m3_d * 2:.0f} m³/d capacity
   - Dewatering: Belt filter press or centrifuge
   - Target cake solids: 25-35%
   - Disposal: Landfill or beneficial reuse
</div>
"""
display(HTML(equipment_specs))

## 10. PROCESS CONTROL STRATEGY

In [ ]:
# Create process control diagram
fig, ax = plt.subplots(figsize=(12, 8))

# Define control loops
control_points = [
    {'name': 'Raw Water pH', 'setpoint': f'{ph_value:.1f}', 'action': 'Monitor only'},
    {'name': 'Rapid Mix pH', 'setpoint': '10.3-10.5', 'action': 'Control lime feed'},
    {'name': 'Clarifier Effluent pH', 'setpoint': '10.0-10.5', 'action': 'Trim lime dose'},
    {'name': 'Effluent Hardness', 'setpoint': '<50 mg/L', 'action': 'Alarm if high'},
    {'name': 'Sludge Blanket Level', 'setpoint': '2-3 m', 'action': 'Control blowdown'}
]

# Create control strategy table
control_df = pd.DataFrame(control_points)
display(HTML("<b>Table 3: Process Control Points</b>"))
display(control_df.style.hide(axis='index').set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('text-align', 'left'), ('padding', '8px')]}
]))

print("\nCONTROL PHILOSOPHY:")
print("1. Primary control: Feed-forward based on influent flow and quality")
print("2. Trim control: Feedback from rapid mix pH (PID control)")
print("3. Cascade control: Master controller on effluent hardness")
print("4. Alarming: High/low pH, high hardness, high sludge blanket")
print("5. Data logging: 15-minute intervals for trends and optimization")

## 11. CONCLUSIONS

Based on the comprehensive analysis including equilibrium modeling and kinetic evaluation:

1. **Lime Dosage**: The recommended lime dose of **{{actual_lime_dose * 74.09:.0f}} mg/L Ca(OH)₂** achieves the target hardness reduction
2. **Water Quality**: Final hardness of **{{final_hardness:.0f}} mg/L as CaCO₃** meets the design criteria of <50 mg/L
3. **Removal Efficiency**: Total hardness removal of **{{hardness_removal:.1f}}%** is achieved
4. **Kinetics**: Minimum detention time of **{{detention_time:.0f}} minutes** required for 90% precipitation completion
5. **Sludge Production**: Estimated at **{{sludge_dry_kg_d:,.0f}} kg/day** dry solids
6. **Operating Cost**: Chemical cost of **${{annual_chem_cost:,.0f}}/year** at design flow

## 12. RECOMMENDATIONS

### 12.1 Design Recommendations
1. **Reactor Configuration**:
   - Two-stage system: rapid mix (3 min) + flocculation ({{detention_time:.0f}} min)
   - Solids contact clarifier for improved efficiency
   - Sludge recirculation for seed crystal benefits

2. **Chemical Feed System**:
   - Dry lime feed with on-site slaking
   - Capacity: {{lime_dose_kg_h * 1.2:.0f}} kg/h (20% safety factor)
   - Automatic pH control with feed-forward/feedback loops

3. **Process Optimization**:
   - Consider split treatment if Mg removal not required
   - Evaluate sludge recycle (5-10%) for nucleation
   - Monitor for post-precipitation in distribution

### 12.2 Operational Recommendations
1. **Monitoring Program**:
   - Continuous: pH, flow, turbidity
   - Daily: hardness, alkalinity, lime purity
   - Weekly: full mineral analysis, sludge characteristics

2. **Optimization Targets**:
   - Maintain rapid mix pH: 10.3-10.5
   - Control effluent pH: 10.0-10.2
   - Minimize excess lime while ensuring compliance

3. **Contingency Planning**:
   - Standby equipment for critical components
   - Emergency lime supply (3-day minimum)
   - Alternative sludge disposal arrangements

## APPENDIX A: SATURATION INDICES

In [ ]:
# Display saturation indices with proper extraction
initial_si = spec_results.get('saturation_indices', {})
final_si = treatment_results.get('saturation_indices', {})

# Common minerals for lime softening
minerals_of_interest = ['Calcite', 'Aragonite', 'Vaterite', 'Brucite', 'Portlandite', 
                       'Dolomite', 'Gypsum', 'Anhydrite']

si_data = []
for mineral in minerals_of_interest:
    initial_val = initial_si.get(mineral, None)
    final_val = final_si.get(mineral, None)
    
    if initial_val is not None or final_val is not None:
        status = 'Precipitating' if final_val and final_val > 0 else 'Undersaturated'
        si_data.append({
            'Mineral': mineral,
            'Formula': {
                'Calcite': 'CaCO₃',
                'Aragonite': 'CaCO₃',
                'Vaterite': 'CaCO₃',
                'Brucite': 'Mg(OH)₂',
                'Portlandite': 'Ca(OH)₂',
                'Dolomite': 'CaMg(CO₃)₂',
                'Gypsum': 'CaSO₄·2H₂O',
                'Anhydrite': 'CaSO₄'
            }.get(mineral, ''),
            'Initial SI': format_number(initial_val, 2, 'N/A'),
            'Final SI': format_number(final_val, 2, 'N/A'),
            'Status': status
        })

if si_data:
    si_df = pd.DataFrame(si_data)
    display(HTML("<b>Table A1: Mineral Saturation Indices</b>"))
    display(si_df.style.hide(axis='index').set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '8px')]}
    ]))
    
    print("\nNOTES:")
    print("- SI > 0: Supersaturated (will precipitate)")
    print("- SI = 0: At equilibrium")
    print("- SI < 0: Undersaturated (will not precipitate)")
    print("- Calcite is the preferred CaCO₃ polymorph in water treatment")
else:
    print("Saturation index data not available.")

## APPENDIX B: CALCULATION VERIFICATION

In [ ]:
# Mass balance verification
print("=== MASS BALANCE VERIFICATION ===")

# Calcium balance
ca_in = ca_mmol * flow_rate * 40.08 / 1000  # kg/h Ca
ca_added = actual_lime_dose * flow_rate * 40.08 / 1000  # kg/h Ca from lime
ca_out = final_ca * flow_rate * 40.08 / 1000  # kg/h Ca
ca_precipitated = (ca_in + ca_added - ca_out)

print(f"\nCALCIUM BALANCE:")
print(f"  Input:         {ca_in:8.2f} kg/h")
print(f"  Added (lime):  {ca_added:8.2f} kg/h")
print(f"  Output:        {ca_out:8.2f} kg/h")
print(f"  Precipitated:  {ca_precipitated:8.2f} kg/h")
print(f"  Balance:       {(ca_in + ca_added - ca_out - ca_precipitated):8.2f} kg/h")

# Magnesium balance
mg_in = mg_mmol * flow_rate * 24.31 / 1000  # kg/h Mg
mg_out = final_mg * flow_rate * 24.31 / 1000  # kg/h Mg
mg_precipitated = mg_in - mg_out

print(f"\nMAGNESIUM BALANCE:")
print(f"  Input:         {mg_in:8.2f} kg/h")
print(f"  Output:        {mg_out:8.2f} kg/h")
print(f"  Precipitated:  {mg_precipitated:8.2f} kg/h")
print(f"  Balance:       {(mg_in - mg_out - mg_precipitated):8.2f} kg/h")

# Alkalinity balance
alk_consumed = (ca_mmol - final_ca) * 2  # 2 moles HCO3- per mole Ca precipitated
print(f"\nALKALINITY BALANCE:")
print(f"  Initial:       {alk_mmol:8.2f} mmol/L")
print(f"  Consumed:      {alk_consumed:8.2f} mmol/L")
print(f"  Theoretical:   {alk_mmol - alk_consumed:8.2f} mmol/L")
print(f"  Actual final:  {final_alk:8.2f} mmol/L")

## APPENDIX C: PHREEQC INPUT FILE

In [ ]:
# Generate example PHREEQC input for documentation
phreeqc_input = f"""
# PHREEQC Input File for Lime Softening Calculation
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

SOLUTION 1 Raw Water
    temp      {temp_c}
    pH        {ph_value}
    units     mmol/L
    Ca        {ca_mmol}
    Mg        {mg_mmol}
    Alkalinity {alk_mmol} as HCO3
    Na        {na_mmol if na_mmol > 0 else 1.0}
    Cl        {cl_mmol if cl_mmol > 0 else 1.0} charge

REACTION 1
    Ca(OH)2   {actual_lime_dose} mmol

EQUILIBRIUM_PHASES 1
    Calcite    0  0
    Brucite    0  0
    CO2(g)     -3.5  10

SAVE solution 2
END

# Note: This is a simplified input file. 
# Actual PHREEQC runs may include additional phases, 
# kinetic blocks, and output specifications.
"""

print("PHREEQC Input File:")
print("=" * 60)
print(phreeqc_input)

---
*End of Calculation Sheet*  
*This document was generated using automated water chemistry modeling tools and should be reviewed by a qualified engineer.*